# Part 1: Coding skills with general data science use case

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=ad5ca079392d31ae5a3321610778a018f3fcc6adcd142b982d30ebb6ed8fff36
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("TestApp").getOrCreate()

In [ ]:
billing = [
        {'Customer': 'Bob', 'Paid Date': '2022-01-31', 'Paid Amount': '100'},
        {'Customer': 'Bob', 'Paid Date': '2022-02-25', 'Paid Amount': '140'},
        {'Customer': 'Bob', 'Paid Date': '2022-03-15', 'Paid Amount': '120'},
        {'Customer': 'Lee', 'Paid Date': '2022-01-15', 'Paid Amount': '150'},
        {'Customer': 'Lee', 'Paid Date': '2022-02-28', 'Paid Amount': '135'},
        {'Customer': 'Nok', 'Paid Date': '2022-04-13', 'Paid Amount': '200'},
       ]

extraBills = [
    {'Customer': 'Nok', 'Paid Date': '2022-05-31', 'Paid Amount': '201'},
    {'Customer': 'Bob', 'Paid Date': '2022-03-25', 'Paid Amount': '160'}
]

### 1. How to filter ONLY rows with max “Paid Date” of each “Customer”.

In [ ]:
billing_df = spark.createDataFrame(billing)
billing_df = billing_df.withColumn("Paid Date", to_date(billing_df["Paid Date"], "yyyy-MM-dd"))

In [ ]:
billing_df.show()

+--------+-----------+----------+
|Customer|Paid Amount| Paid Date|
+--------+-----------+----------+
|     Bob|        100|2022-01-31|
|     Bob|        140|2022-02-25|
|     Bob|        120|2022-03-15|
|     Lee|        150|2022-01-15|
|     Lee|        135|2022-02-28|
|     Nok|        200|2022-04-13|
+--------+-----------+----------+



In [ ]:
q1_max_date = billing_df.groupBy("Customer").agg(max("Paid Date").alias("Max Paid Date")).withColumnRenamed('Customer', 'Cust')

q1_max_date = q1_max_date.join(billing_df, (billing_df["Customer"] == q1_max_date["Cust"]) & (billing_df["Paid Date"] == q1_max_date["Max Paid Date"]), how='inner')\
              .select('Cust', 'Max Paid Date', 'Paid Amount').withColumnRenamed('Cust', 'Customer').withColumnRenamed('Max Paid Date','Paid Date')

In [ ]:
q1_max_date.show()

+--------+----------+-----------+
|Customer| Paid Date|Paid Amount|
+--------+----------+-----------+
|     Bob|2022-03-15|        120|
|     Nok|2022-04-13|        200|
|     Lee|2022-02-28|        135|
+--------+----------+-----------+



### 2. How to combine the “Billing” DataFrame with “ExtraBills” as below:

In [ ]:
extra_billing_df = spark.createDataFrame(extraBills)
extra_billing_df = extra_billing_df.withColumn("Paid Date", to_date(extra_billing_df["Paid Date"], "yyyy-MM-dd"))

In [ ]:
extra_billing_df.show()

+--------+-----------+----------+
|Customer|Paid Amount| Paid Date|
+--------+-----------+----------+
|     Nok|        201|2022-05-31|
|     Bob|        160|2022-03-25|
+--------+-----------+----------+



In [ ]:
billing_concat = billing_df.unionByName(extra_billing_df)
billing_concat.show()

+--------+-----------+----------+
|Customer|Paid Amount| Paid Date|
+--------+-----------+----------+
|     Bob|        100|2022-01-31|
|     Bob|        140|2022-02-25|
|     Bob|        120|2022-03-15|
|     Lee|        150|2022-01-15|
|     Lee|        135|2022-02-28|
|     Nok|        200|2022-04-13|
|     Nok|        201|2022-05-31|
|     Bob|        160|2022-03-25|
+--------+-----------+----------+



### 3. Find the number of days between “Paid Date” of each “Customer” from “Billing”.

In [ ]:
windowSpec = Window.partitionBy("Customer").orderBy("Paid Date")
between_date = billing_df.withColumn("Days Difference", datediff(col("Paid Date"), lag(col("Paid Date")).over(windowSpec)))

In [ ]:
between_date.show()

+--------+-----------+----------+---------------+
|Customer|Paid Amount| Paid Date|Days Difference|
+--------+-----------+----------+---------------+
|     Bob|        100|2022-01-31|           NULL|
|     Bob|        140|2022-02-25|             25|
|     Bob|        120|2022-03-15|             18|
|     Lee|        150|2022-01-15|           NULL|
|     Lee|        135|2022-02-28|             44|
|     Nok|        200|2022-04-13|           NULL|
+--------+-----------+----------+---------------+

